In [5]:
import os
import random
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.applications import ResNet50
from keras.callbacks import TensorBoard


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
train_data_dir = '/path/to/train/data'
val_data_dir = '/path/to/validation/data'
test_data_dir = '/path/to/test/data'


In [ ]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 50

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=10,
    fill_mode='nearest')

train_crop_generator = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=10,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255,
                                 preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

test_datagen = ImageDataGenerator(rescale=1./255,
                                  preprocessing_function=tf.keras.applications.resnet50.preprocess_input)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='binary',
    shuffle=False)


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Add new classification layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
logdir = os.path.join("logs", "resnet50")
tensorboard_callback = TensorBoard(log_dir=logdir)


In [ ]:
model.fit(train_generator,
          steps_per_epoch=train_generator.samples//batch_size,
          validation_data=val_generator,
          validation_steps=val_generator.samples//batch_size,
          epochs=epochs,
          callbacks=[tensorboard_callback])


In [ ]:
test_generator.reset()
predictions = []
for i in range(test_generator.samples):
    image = test_generator.next()[0]
    crops = train_crop_generator.random_transform(image, seed=i)
    crop_predictions = []
    for crop in crops:
        crop = np.expand_dims(crop, axis=0)
        crop_prediction = model.predict(crop)
        crop_predictions.append(crop_prediction[0])
    majority_vote = np.round(np.mean(crop_predictions))
    predictions.append(majority_vote)


In [ ]:
accuracy = np.sum(predictions == test_generator.labels)/test_generator.samples
print(f"Test accuracy: {accuracy}")


In [ ]:
model.save("resnet50.h5")


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs
